In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
%matplotlib inline 
# to put the graphs directly inder the cells instead of in a pop-up

In [11]:
# 1. data loading

# column definition following the NASA C-MAPSS documentation
index_names = ['unit_nr', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['sensor_{}'.format(i) for i in range(1, 22)] 
col_names = index_names + setting_names + sensor_names

# data loading
print("Loading data from dataset...")
df_train = pd.read_csv('data/train_FD001.txt', sep='\s+', header=None, names=col_names)
df_test = pd.read_csv('data/test_FD001.txt', sep='\s+', header=None, names=col_names)
y_test = pd.read_csv('data/RUL_FD001.txt', sep='\s+', header=None, names=['RUL'])

print("Data loaded successfully!")
print(f"Training set shape: {df_train.shape}")
print(f"Test set shape: {df_test.shape}")

# preview on the five first lines
df_train.head()

<>:11: SyntaxWarning: invalid escape sequence '\s'
<>:12: SyntaxWarning: invalid escape sequence '\s'
<>:13: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\s'
<>:12: SyntaxWarning: invalid escape sequence '\s'
<>:13: SyntaxWarning: invalid escape sequence '\s'
C:\Users\_\AppData\Local\Temp\ipykernel_13188\281608674.py:11: SyntaxWarning: invalid escape sequence '\s'
  df_train = pd.read_csv('data/train_FD001.txt', sep='\s+', header=None, names=col_names)
C:\Users\_\AppData\Local\Temp\ipykernel_13188\281608674.py:12: SyntaxWarning: invalid escape sequence '\s'
  df_test = pd.read_csv('data/test_FD001.txt', sep='\s+', header=None, names=col_names)
C:\Users\_\AppData\Local\Temp\ipykernel_13188\281608674.py:13: SyntaxWarning: invalid escape sequence '\s'
  y_test = pd.read_csv('data/RUL_FD001.txt', sep='\s+', header=None, names=['RUL'])


Loading data from dataset...
Data loaded successfully!
Training set shape: (20631, 26)
Test set shape: (13096, 26)


,unit_nr,time_cycles,setting_1,setting_2,setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
